In [27]:
import cv2
import torch
import numpy as np
import os
from torch.utils.data import TensorDataset, DataLoader
import util
import torch.nn.functional as F
     
 
patient_ID = 15           # up to 16                                   

  

Now draw ribs

In [28]:
util.popupmsg('draw ribs now')

def path_prep_data(rootpath, patient_ID):
    patient_ID = str(patient_ID)
    
    idx_path = os.path.join(rootpath ,"p" + patient_ID + "t2", "p" + patient_ID + "t2ct1_raw")

    return idx_path

def click_event(event, x, y, flags, params):
 
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN or event==cv2.EVENT_RBUTTONDOWN:       
        cv2.circle(img, (x,y), radius=0, color=(255, 0, 0), thickness=1)
        if len(pts) >= 1:
            cv2.line(img, pts[-1], (x,y), color=(255, 0, 0), thickness=1)
        cv2.imshow(str(ind), img)
        cv2.moveWindow(str(ind), 40,30)
        pts.append([x,y])
   
# driver function

 
# reading the image


# patient_ID_list = [4,1,6,17]
rootpath = os.path.join(os.path.dirname(os.getcwd()),'python')

idx_path = path_prep_data(rootpath, patient_ID)
 
# path
path = idx_path+'.npy'

idx = np.load(idx_path + '.npy')

idx = idx > 200
 
# Reading an image in default
# mode
# image = idx
under_factor = 5
ind = 0
cnt = 0
roi = np.zeros([idx.shape[0],idx.shape[1]//under_factor+1,idx.shape[2]])
while ind < idx.shape[1]:
    img = idx[:,ind,:]
    img = np.stack([img,img,img],2)
    img = img.astype('float64')


    # displaying the image
    cv2.imshow(str(ind), img)
    cv2.moveWindow(str(ind), 40,30)

    pts = []

    # setting mouse handler for the image
    # and calling the click_event() function
    cv2.setMouseCallback(str(ind), click_event)

    # wait for a key to be pressed to exit
    cv2.waitKey(0)

    # close the window
    cv2.destroyAllWindows()

    window_name = str(ind)

    isClosed = True
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.polylines() method
    # Draw a Blue polygon with
    # thickness of 1 px
    pts = np.array(pts,np.int32)
    pts = pts.reshape((-1, 1, 2))


    image = cv2.polylines(img, [pts],
                        isClosed, color, thickness)
    
    # Displaying the image      
    cv2.imshow(str(ind), image)
    cv2.moveWindow(str(ind), 40,30)   
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    mask = cv2.fillConvexPoly(img, pts,color)
    mask = mask[:,:,0] > 100
    mask = mask.astype(int)

    roi[:,cnt,:] = mask

    ind += under_factor
    cnt+=1
roi_tensor = torch.tensor(roi).unsqueeze(0).unsqueeze(0)
roi_tensor_interpolate = F.interpolate(roi_tensor, size=(idx.shape[0],idx.shape[1]//under_factor * under_factor, idx.shape[2]), mode='trilinear', align_corners=True).squeeze(0).squeeze(0).numpy()
roi_temp= np.zeros_like(idx)
roi_temp[:,:roi_tensor_interpolate.shape[1],:] = roi_tensor_interpolate
roi_interpolate = roi_temp.astype(int)
roi_bone = roi_interpolate * idx
ind = 0
# while ind < idx.shape[1]:
#     img = roi_bone[:,ind,:]
#     img = np.stack([img,img,img],2)
#     img = img.astype('float64')


#     # displaying the image
#     cv2.imshow(str(ind), img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     ind += 1
# util.pvplotgif(roi_bone*1000, filename='rib.tif', title='rib',colormap='bone')
roi_rib = roi_bone

Now select upper spine

In [29]:
util.popupmsg('draw upper spine now')

under_factor = 5
ind = 0
cnt = 0
roi = np.zeros([idx.shape[0],idx.shape[1]//under_factor+1,idx.shape[2]])
while ind < idx.shape[1]:
    img = idx[:,ind,:]
    img = np.stack([img,img,img],2)
    img = img.astype('float64')


    # displaying the image
    cv2.imshow(str(ind), img)
    cv2.moveWindow(str(ind), 40,30)

    pts = []

    # setting mouse handler for the image
    # and calling the click_event() function
    cv2.setMouseCallback(str(ind), click_event)

    # wait for a key to be pressed to exit
    cv2.waitKey(0)

    # close the window
    cv2.destroyAllWindows()

    
    # Window name in which image is
    # displayed
    window_name = str(ind)


    isClosed = True
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.polylines() method
    # Draw a Blue polygon with
    # thickness of 1 px
    pts = np.array(pts,np.int32)
    pts = pts.reshape((-1, 1, 2))


    image = cv2.polylines(img, [pts],
                        isClosed, color, thickness)
    
    # Displaying the image      
    cv2.imshow(str(ind), image)
    cv2.moveWindow(str(ind), 40,30)   
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    mask = cv2.fillConvexPoly(img, pts,color)
    mask = mask[:,:,0] > 100
    mask = mask.astype(int)

    roi[:,cnt,:] = mask

    ind += under_factor
    cnt+=1
roi_tensor = torch.tensor(roi).unsqueeze(0).unsqueeze(0)
roi_tensor_interpolate = F.interpolate(roi_tensor, size=(idx.shape[0],idx.shape[1]//under_factor * under_factor, idx.shape[2]), mode='trilinear', align_corners=True).squeeze(0).squeeze(0).numpy()
roi_temp= np.zeros_like(idx)
roi_temp[:,:roi_tensor_interpolate.shape[1],:] = roi_tensor_interpolate
roi_interpolate = roi_temp.astype(int)
roi_bone = roi_interpolate * idx
# ind = 0
# while ind < idx.shape[1]:
#     img = roi_bone[:,ind,:]
#     img = np.stack([img,img,img],2)
#     img = img.astype('float64')


#     # displaying the image
#     cv2.imshow(str(ind), img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     ind += 1

# util.pvplotgif(roi_bone*1000, filename='upperSpine.tif', title='upper spine',colormap='bone')
roi_upperspine = roi_bone

Now select mid spine

In [30]:
util.popupmsg('draw mid spine now')
under_factor = 5
ind = 0
cnt = 0
roi = np.zeros([idx.shape[0],idx.shape[1]//under_factor+1,idx.shape[2]])
while ind < idx.shape[1]:
    img = idx[:,ind,:]
    img = np.stack([img,img,img],2)
    img = img.astype('float64')


    # displaying the image
    cv2.imshow(str(ind), img)
    cv2.moveWindow(str(ind), 40,30)

    pts = []

    # setting mouse handler for the image
    # and calling the click_event() function
    cv2.setMouseCallback(str(ind), click_event)

    # wait for a key to be pressed to exit
    cv2.waitKey(0)

    # close the window
    cv2.destroyAllWindows()

    
    # Window name in which image is
    # displayed
    window_name = str(ind)


    isClosed = True
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.polylines() method
    # Draw a Blue polygon with
    # thickness of 1 px
    pts = np.array(pts,np.int32)
    pts = pts.reshape((-1, 1, 2))


    image = cv2.polylines(img, [pts],
                        isClosed, color, thickness)
    
    # Displaying the image      
    cv2.imshow(str(ind), image)
    cv2.moveWindow(str(ind), 40,30)   
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    mask = cv2.fillConvexPoly(img, pts,color)
    mask = mask[:,:,0] > 100
    mask = mask.astype(int)

    roi[:,cnt,:] = mask

    ind += under_factor
    cnt+=1
roi_tensor = torch.tensor(roi).unsqueeze(0).unsqueeze(0)
roi_tensor_interpolate = F.interpolate(roi_tensor, size=(idx.shape[0],idx.shape[1]//under_factor * under_factor, idx.shape[2]), mode='trilinear', align_corners=True).squeeze(0).squeeze(0).numpy()
roi_temp= np.zeros_like(idx)
roi_temp[:,:roi_tensor_interpolate.shape[1],:] = roi_tensor_interpolate
roi_interpolate = roi_temp.astype(int)
roi_bone = roi_interpolate * idx
# ind = 0
# while ind < idx.shape[1]:
#     img = roi_bone[:,ind,:]
#     img = np.stack([img,img,img],2)
#     img = img.astype('float64')


#     # displaying the image
#     cv2.imshow(str(ind), img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     ind += 1

# util.pvplotgif(roi_bone*1000, filename='midspine.tif', title='mid spine',colormap='bone')
roi_midspine = roi_bone

Now select lower spine

In [31]:
util.popupmsg('draw lower spine now')
under_factor = 5
ind = 0
cnt = 0
roi = np.zeros([idx.shape[0],idx.shape[1]//under_factor+1,idx.shape[2]])
while ind < idx.shape[1]:
    img = idx[:,ind,:]
    img = np.stack([img,img,img],2)
    img = img.astype('float64')


    # displaying the image
    cv2.imshow(str(ind), img)
    cv2.moveWindow(str(ind), 40,30)

    pts = []

    # setting mouse handler for the image
    # and calling the click_event() function
    cv2.setMouseCallback(str(ind), click_event)

    # wait for a key to be pressed to exit
    cv2.waitKey(0)

    # close the window
    cv2.destroyAllWindows()

    
    # Window name in which image is
    # displayed
    window_name = str(ind)


    isClosed = True
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.polylines() method
    # Draw a Blue polygon with
    # thickness of 1 px
    pts = np.array(pts,np.int32)
    pts = pts.reshape((-1, 1, 2))


    image = cv2.polylines(img, [pts],
                        isClosed, color, thickness)
    
    # Displaying the image      
    cv2.imshow(str(ind), image) 
    cv2.moveWindow(str(ind), 40,30)  
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    mask = cv2.fillConvexPoly(img, pts,color)
    mask = mask[:,:,0] > 100
    mask = mask.astype(int)

    roi[:,cnt,:] = mask

    ind += under_factor
    cnt+=1
roi_tensor = torch.tensor(roi).unsqueeze(0).unsqueeze(0)
roi_tensor_interpolate = F.interpolate(roi_tensor, size=(idx.shape[0],idx.shape[1]//under_factor * under_factor, idx.shape[2]), mode='trilinear', align_corners=True).squeeze(0).squeeze(0).numpy()
roi_temp= np.zeros_like(idx)
roi_temp[:,:roi_tensor_interpolate.shape[1],:] = roi_tensor_interpolate
roi_interpolate = roi_temp.astype(int)
roi_bone = roi_interpolate * idx
roi_lowerspine = roi_bone
# util.pvplotgif(roi_bone*1000, filename='lowerspine.tif', title='lower spine',colormap='bone')


Now select pelvis

In [32]:
util.popupmsg('draw pelvis now')
under_factor = 5
ind = 0
cnt = 0
roi = np.zeros([idx.shape[0],idx.shape[1]//under_factor+1,idx.shape[2]])
while ind < idx.shape[1]:
    img = idx[:,ind,:]
    img = np.stack([img,img,img],2)
    img = img.astype('float64')


    # displaying the image
    cv2.imshow(str(ind), img)
    cv2.moveWindow(str(ind), 40,30)

    pts = []

    # setting mouse handler for the image
    # and calling the click_event() function
    cv2.setMouseCallback(str(ind), click_event)

    # wait for a key to be pressed to exit
    cv2.waitKey(0)

    # close the window
    cv2.destroyAllWindows()

    
    # Window name in which image is
    # displayed
    window_name = str(ind)


    isClosed = True
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.polylines() method
    # Draw a Blue polygon with
    # thickness of 1 px
    pts = np.array(pts,np.int32)
    pts = pts.reshape((-1, 1, 2))


    image = cv2.polylines(img, [pts],
                        isClosed, color, thickness)
    
    # Displaying the image      
    cv2.imshow(str(ind), image)   
    cv2.moveWindow(str(ind), 40,30)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    mask = cv2.fillConvexPoly(img, pts,color)
    mask = mask[:,:,0] > 100
    mask = mask.astype(int)

    roi[:,cnt,:] = mask

    ind += under_factor
    cnt+=1
roi_tensor = torch.tensor(roi).unsqueeze(0).unsqueeze(0)
roi_tensor_interpolate = F.interpolate(roi_tensor, size=(idx.shape[0],idx.shape[1]//under_factor * under_factor, idx.shape[2]), mode='trilinear', align_corners=True).squeeze(0).squeeze(0).numpy()
roi_temp= np.zeros_like(idx)
roi_temp[:,:roi_tensor_interpolate.shape[1],:] = roi_tensor_interpolate
roi_interpolate = roi_temp.astype(int)
roi_bone = roi_interpolate * idx
roi_pelvis = roi_bone
# util.pvplotgif(roi_bone*1000, filename='pelvis.tif', title='pelvis',colormap='bone')


In [33]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np

import pyvista as pv


roi_rib, roi_upperspine,roi_midspine,roi_lowerspine, roi_pelvis= np.ceil(roi_rib),np.ceil(roi_upperspine),np.ceil(roi_midspine), np.ceil(roi_lowerspine), np.ceil(roi_pelvis)
roi_all = np.stack([roi_rib,roi_upperspine,roi_midspine,roi_lowerspine, roi_pelvis],3)

roi_rib[(roi_rib + roi_midspine) > 1] = 1
roi_upperspine[(roi_upperspine + roi_midspine) > 1] = 1
roi_midspine[(roi_lowerspine + roi_midspine) > 1] = 1
roi_pelvis[(roi_lowerspine + roi_pelvis) > 1] = 1


roi_total = roi_rib + 2*roi_upperspine + 3*roi_midspine + 4*roi_lowerspine + 5*roi_pelvis
roi_name = idx_path+'_roi.npy'
# np.save(roi_name, roi_total)
np.save(roi_name, roi_all)

roi_total_1 = roi_total * 50
print(roi_total_1.max())
# roi_total_1[roi_total_1 > 250] = 0


# roi_total_1[roi_total_1 != 250] = 0
plotter = pv.Plotter()
pvplot = pv.wrap(roi_total_1)
light = pv.Light(position=(-1, 1, 1), color='white')
light.positional = True
# Define the colors we want to use
blue = np.array([12 / 256, 238 / 256, 246 / 256, 1.0])
black = np.array([11 / 256, 11 / 256, 11 / 256, 1.0])
grey = np.array([189 / 256, 189 / 256, 189 / 256, 1.0])
yellow = np.array([255 / 256, 247 / 256, 0 / 256, 1.0])
red = np.array([1.0, 0.0, 0.0, 1.0])
green = np.array([50/ 256, 200 / 256, 50 / 256, 1.0])

# mapping = np.linspace(roi_total_1.min(), roi_total_1.max(), 250)
mapping = np.linspace(0, 256, 256)
newcolors = np.empty((256, 4))


newcolors[mapping <= 250] = red
newcolors[mapping <= 200] = green
newcolors[mapping <= 150] = yellow
newcolors[mapping <= 100] = blue
newcolors[mapping <50] = black



my_colormap = ListedColormap(newcolors)
plotter.add_volume(pvplot, cmap=my_colormap,blending='additive', shade=True, specular=1, ambient=1)
plotter.camera_position = 'yz'
plotter.add_light(light)
plotter.show(auto_close=False)

# util.pvplotgif(roi_total*1000, filename='total.tif', title='total',colormap='coolwarm')

250.0


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)